In [9]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
pd.set_option('display.max_columns', None)  
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler, scale
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Add, LSTM, Activation, Flatten, Dropout, Lambda, Conv1D, Input, BatchNormalization
from keras.models import Model
import numpy as np
from keras.utils import to_categorical
import fix_yahoo_finance as yf
%matplotlib inline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')
import glob

In [82]:
stocks1=['OI','NI','UPS','HSIC','CCE','IBM','MAS','EFX','AAPL','WDC']
stocks2=['AKS','AEG','BCS','BBVA','CBL','CPE','ERIC']
stocks=stocks1+stocks2
stocks=glob.glob('*.csv')
for stock in stocks:
    df = yf.download(stock.split('.')[0],'2001-10-06','2019-02-11')
    df.to_csv(stock)
    print(stock)
#df=pd.read_csv(Stock+'.csv')
#df.tail()

[*********************100%***********************]  1 of 1 downloaded
AAPL.csv
[*********************100%***********************]  1 of 1 downloaded
ACB.csv
[*********************100%***********************]  1 of 1 downloaded
ADBE.csv
[*********************100%***********************]  1 of 1 downloaded
AEG.csv
[*********************100%***********************]  1 of 1 downloaded
AG.csv
[*********************100%***********************]  1 of 1 downloaded
AKS.csv
[*********************100%***********************]  1 of 1 downloaded
AMAT.csv
[*********************100%***********************]  1 of 1 downloaded
AMD.csv
[*********************100%***********************]  1 of 1 downloaded
AMZN.csv
[*********************100%***********************]  1 of 1 downloaded
AST.csv
[*********************100%***********************]  1 of 1 downloaded
BAC.csv
[*********************100%***********************]  1 of 1 downloaded
BBVA.csv
[*********************100%***********************]  1 of 1 d

In [84]:
def data_prepare(Stock):
    df=pd.read_csv(Stock+'.csv')
    data=df.copy()
    data['Close'].replace(0, np.nan, inplace=True)
    data['Close'].fillna(method='ffill', inplace=True)
    values = data[['Open'] + ['High'] + ['Low'] + ['Close'] + ['Volume']]
    values.dropna(inplace=True)
    values = values.astype('float32')
    values=values[(values['Open']!=0)&(values['High']!=0)&(values['Low']!=0)&(values['Close']!=0)&(values['Volume']!=0)]
    Y = values.iloc[1:,3] - values.iloc[1:,0]
    scaled = (values.values[1:,:]/values.values[:-1,:])
    scaled = pd.DataFrame(scaled)
    all_inf_or_nan = scaled.isin([np.inf, -np.inf, np.nan]).all(axis='columns')
    scaled = scaled[~all_inf_or_nan]
    scaled = scaled.values
    return scaled, Y

In [85]:
def data_gen(scaled, Y, end, seq_len):
    X = scaled[-(seq_len+1):-1,:].reshape(1,seq_len,5)
    Y = np.array(Y)
    y = Y[-1]
    for i in range(1,len(scaled)-seq_len):
        X = np.append(X, scaled[-(seq_len+1)-i:-1-i,:].reshape(1,seq_len,5),axis=0)
        #y = np.append(y, scaled[-1-i,3])
        y = np.append(y, Y[-1-i])
    X = X[::-1,:,:]
    y = y[::-1]
    y1 = np.empty([len(y)], dtype=np.float32)
    for i in range(len(y)):
        if y[i] >= 0.0:
            y1[i] = 1.0
        else:
            y1[i] = 0.0
    #y1=to_categorical(y1)
    if end==0:
        train_X = X
        train_y = y1
        train_X, train_y = shuffle(train_X, train_y, random_state = 128)
        return train_X, train_y
    else:
        train_X = X[:-end]
        train_y = y1[:-end]
        train_X, train_y = shuffle(train_X, train_y, random_state = 128)
        train_X = X[:-end]
        train_y = y1[:-end]
        train_X, train_y = shuffle(train_X, train_y, random_state = 128)
        test_X = X[-end:]
        test_y = y1[-end:]
        return train_X, train_y, test_X, test_y


In [69]:
model = XGBClassifier(max_depth=3, n_estimators=100)
model.fit(train_X.reshape(4107,seq_len*5), train_y)
pred_y = model.predict(test_X.reshape(20,seq_len*5))
accuracy = accuracy_score(test_y,pred_y)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 60.00%


C:\Users\alahghol\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [49]:
#clf = RandomForestClassifier(n_estimators=250, max_depth=80,random_state=0)
#clf.fit(train_X.reshape(4265,75), train_y)
N = KNeighborsClassifier(n_neighbors=685)
N.fit(train_X.reshape(4107,seq_len*5), train_y)
N.score(test_X.reshape(20,seq_len*5),test_y)

0.7

In [86]:
#stocks=['OI','NI','UPS','HSIC','IBM','MAS','EFX','AAPL','WDC']
stocks=['AKS','AEG','BCS','BBVA','CBL','CPE','ERIC']
stocks=glob.glob('*.csv')
stocks=[i.split('.')[0] for i in stocks]
accuracy=[]
accuracy_ind=[]
predict=[]
seq_len=15
for stock in stocks:
    score=[]
    pred=[]
    scaled, Y = data_prepare(stock)
    train_X, train_y, test_X, test_y = data_gen(scaled, Y, 20, seq_len)
    for n in range(100,1000,100):
        N = KNeighborsClassifier(n_neighbors=n)
        N.fit(train_X.reshape(len(train_X),seq_len*5), train_y)
        score.append(N.score(test_X.reshape(20,seq_len*5),test_y))
        pred.append(N.predict(scaled[-seq_len:,:].reshape(1,seq_len*5)))
    accuracy.append(max(score))
    accuracy_ind.append((score.index(max(score))+1)*100)
    predict.append(pred[score.index(max(score))])
    print(stock, accuracy_ind[-1], accuracy[-1])
    score=[]
    pred=[]
    for n in range(accuracy_ind[-1]-99,accuracy_ind[-1]+100,10):
        N = KNeighborsClassifier(n_neighbors=n)
        N.fit(train_X.reshape(len(train_X),seq_len*5), train_y)
        score.append(N.score(test_X.reshape(20,seq_len*5),test_y))
        pred.append(N.predict(scaled[-seq_len:,:].reshape(1,seq_len*5)))
    accuracy[-1] = (max(score))
    accuracy_ind[-1] = (accuracy_ind[-1]+(score.index(max(score))+1)*10)
    predict[-1] = pred[score.index(max(score))]
accuracy=pd.DataFrame(np.array(accuracy).reshape(1,len(accuracy)),columns=stocks)
accuracy_ind=pd.DataFrame(np.array(accuracy_ind).reshape(1,len(accuracy_ind)),columns=stocks)
predict=pd.DataFrame(np.array(predict).reshape(1,len(predict)),columns=stocks)
accuracy=pd.concat([accuracy,accuracy_ind,predict],axis=0)
accuracy

AAPL 400 0.7
ACB 200 0.55
ADBE 100 0.6
AEG 500 0.7
AG 700 0.7
AKS 300 0.55
AMAT 200 0.6
AMD 600 0.55
AMZN 500 0.55
AST 100 0.4
BAC 400 0.6
BBVA 300 0.85
BCS 900 0.9
BTX 100 0.65
CBL 200 0.65
CCF 400 0.6
COG 300 0.55
CPE 400 0.6
DIA 300 0.75
EFX 400 0.75
ERIC 800 0.5
FB 400 0.85
GE 100 0.5
GOLD 400 0.5
GOOGl 100 0.5
GS 100 0.6
HSIC 100 0.6
IBM 400 0.75
INTC 100 0.8
IO 300 0.5
MAC 100 0.55
MAS 900 0.65
MSFT 600 0.75
MU 800 0.75
NFLX 500 0.7
NI 200 0.55
NOG 100 0.55
NVDA 300 0.7
OI 300 0.75
OSN 500 0.55
QQQ 500 0.65
SCCO 700 0.7
SNE 600 0.65
SPY 400 0.65
TGT 300 0.65
TSM 300 0.7
TVIX 100 0.55
TWTR 400 0.55
UBS 300 0.8
UPS 800 0.8
USO 400 0.8
WDC 400 0.55
WFC 200 0.55
WRN 100 0.55


,AAPL,ACB,ADBE,AEG,AG,AKS,AMAT,AMD,AMZN,AST,BAC,BBVA,BCS,BTX,CBL,CCF,COG,CPE,DIA,EFX,ERIC,FB,GE,GOLD,GOOGl,GS,HSIC,IBM,INTC,IO,MAC,MAS,MSFT,MU,NFLX,NI,NOG,NVDA,OI,OSN,QQQ,SCCO,SNE,SPY,TGT,TSM,TVIX,TWTR,UBS,UPS,USO,WDC,WFC,WRN
0,0.75,0.6,0.6,0.7,0.75,0.6,0.65,0.6,0.55,0.5,0.65,0.85,0.9,0.7,0.75,0.7,0.6,0.65,0.75,0.75,0.5,0.85,0.7,0.5,0.65,0.6,0.65,0.75,0.9,0.55,0.55,0.65,0.7,0.8,0.7,0.6,0.55,0.75,0.75,0.65,0.65,0.7,0.6,0.65,0.65,0.75,0.65,0.6,0.8,0.75,0.8,0.65,0.65,0.85
0,480.00,330.0,200.0,510.0,790.00,360.0,360.00,710.0,510.00,180.0,470.00,410.00,1010.0,120.0,240.00,530.0,410.0,490.00,320.00,420.00,840.0,460.00,270.0,510.0,110.00,290.0,140.00,420.00,180.0,460.00,200.00,940.00,710.0,850.0,600.0,280.0,200.00,420.00,360.00,550.00,580.00,780.0,680.0,490.00,350.00,390.00,110.00,500.0,360.0,810.00,510.0,450.00,380.00,110.00
0,1.00,0.0,1.0,1.0,0.00,0.0,0.00,1.0,1.00,1.0,0.00,1.00,0.0,1.0,1.00,0.0,0.0,1.00,1.00,1.00,1.0,1.00,0.0,1.0,1.00,1.0,1.00,1.00,1.0,0.00,1.00,1.00,0.0,0.0,1.0,1.0,1.00,0.00,0.00,1.00,1.00,1.0,1.0,1.00,0.00,1.00,1.00,1.0,1.0,1.00,1.0,1.00,0.00,1.00


In [87]:
accuracyT=accuracy.transpose()
accuracyT.columns=['accuracy','index','predict']
accuracyT.sort_values(['accuracy'], ascending=False)

,accuracy,index,predict
INTC,0.90,180.0,1.0
BCS,0.90,1010.0,0.0
WRN,0.85,110.0,1.0
FB,0.85,460.0,1.0
BBVA,0.85,410.0,1.0
USO,0.80,510.0,1.0
UBS,0.80,360.0,1.0
MU,0.80,850.0,0.0
DIA,0.75,320.0,1.0
UPS,0.75,810.0,1.0


In [81]:
stock=['USO']
seq_len=15
scaled, Y = data_prepare(stock[0])
train_X, train_y = data_gen(scaled, Y, 0, seq_len)
N = KNeighborsClassifier(n_neighbors=5)
N.fit(train_X.reshape(len(train_X),seq_len*5), train_y)
N.predict(scaled[-seq_len:,:].reshape(1,seq_len*5))

array([1.], dtype=float32)

In [59]:
#stocks=['OI','NI','UPS','HSIC','IBM','MAS','EFX','AAPL','WDC']
stocks=['AKS','AEG','BCS','BBVA','CBL','CPE','ERIC']
accuracy=[]
accuracy_ind=[]
seq_len=5
score=[]
for stock in stocks:
    scaled, Y = data_prepare(stock)
    train_X, train_y, test_X, test_y = data_gen(scaled, Y, 20, seq_len)
    model = XGBClassifier(max_depth=5, n_estimators=100)
    model.fit(train_X.reshape(len(train_X),seq_len*5), train_y)
    pred_y = model.predict(test_X.reshape(20,seq_len*5))
    score.append(accuracy_score(test_y,pred_y))
    #accuracy.append(max(score))
    #accuracy_ind.append((score.index(max(score))+1)*100)
    print(stock, score[-1])
score=pd.DataFrame(np.array(score).reshape(1,len(score)),columns=stocks)
#accuracy_ind=pd.DataFrame(np.array(accuracy_ind).reshape(1,len(accuracy_ind)),columns=stocks)
#accuracy=pd.concat([accuracy,accuracy_ind],axis=0)
score

AKS 0.35
AEG 0.45
BCS 0.7
BBVA 0.75
CBL 0.45
CPE 0.45
ERIC 0.65


,AKS,AEG,BCS,BBVA,CBL,CPE,ERIC
0,0.35,0.45,0.7,0.75,0.45,0.45,0.65


In [66]:
stock=['BBVA']
seq_len=15
scaled, Y = data_prepare(stock[0])
train_X, train_y = data_gen(scaled, Y, 0, seq_len)
model = XGBClassifier(max_depth=5, n_estimators=20)
model.fit(train_X.reshape(len(train_X),seq_len*5), train_y)
model.predict(scaled[-seq_len:,:].reshape(1,seq_len*5))

array([1.], dtype=float32)